# Create Model Description of A Cellpose Model

In [1]:
from pathlib import Path
from bioimageio.spec.common import ValidationError
from bioimageio.spec.model.v0_5 import ModelDescr

To see what does a `ModelDescr` need:

In [2]:
try:
    my_model_descr = ModelDescr()  # type: ignore
except ValidationError as e:
    print(e)

9 validation errors for bioimage.io model specification
name
  Field required [type=missing, input_value={'format_version': '0.5.0', 'type': 'model'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
description
  Field required [type=missing, input_value={'format_version': '0.5.0', 'type': 'model'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
authors
  Field required [type=missing, input_value={'format_version': '0.5.0', 'type': 'model'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
cite
  Field required [type=missing, input_value={'format_version': '0.5.0', 'type': 'model'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
license
  Field required [type=missing, input_value={'format_version': '0.5.0', 'type': 'model'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v

In [3]:
from bioimageio.spec.model.v0_5 import (
    Author,
    AxisId,
    ChannelAxis,
    CiteEntry,
    Doi,
    FileDescr,
    Identifier,
    InputTensorDescr,
    IntervalOrRatioDataDescr,
    ModelDescr,
    OutputTensorDescr,
    PytorchStateDictWeightsDescr,
    SizeReference,
    SpaceInputAxis,
    SpaceOutputAxis,
    TensorId,
    TorchscriptWeightsDescr,
    WeightsDescr,
)

### Input description

In [4]:
input_axes = [
    SpaceInputAxis(id=AxisId("z"), size=32),
    ChannelAxis(channel_names=[Identifier("c1"), Identifier("c2")]),
    SpaceInputAxis(id=AxisId('y'), size=224),
    SpaceInputAxis(id=AxisId('x'), size=224),
]

In [5]:
data_descr = IntervalOrRatioDataDescr(type='float32')

In [6]:
test_input_path = Path('test_input.npy')

In [7]:
input_descr = InputTensorDescr(
    id=TensorId("raw"),
    axes=input_axes,
    test_tensor=FileDescr(source=test_input_path),
    data=data_descr,
)

### Output description

In [8]:
output_axes = [
    SpaceOutputAxis(id=AxisId("z"), size=SizeReference(tensor_id=TensorId('raw'), axis_id=AxisId('z'))),
    ChannelAxis(channel_names=[Identifier("flow1"), Identifier("flow2"), Identifier("flow3")]),
    SpaceOutputAxis(id=AxisId("y"), size=SizeReference(tensor_id=TensorId('raw'), axis_id=AxisId('y'))),
    SpaceOutputAxis(id=AxisId("x"), size=SizeReference(tensor_id=TensorId('raw'), axis_id=AxisId('x'))),
]

In [9]:
test_output_path = Path('test_output.npy')

In [10]:
output_descr = OutputTensorDescr(
    id=TensorId("flow"),
    axes=output_axes,
    test_tensor=FileDescr(source=test_output_path),
)

### Model architecture

In [11]:
from bioimageio.spec.model.v0_5 import (
    ArchitectureFromLibraryDescr,
    Version,
)

In [12]:
try:
    import torch
except ImportError:
    pytorch_version = Version("1.15")
else:
    pytorch_version = Version(torch.__version__)

pytorch_version

Version(root='2.3.0')

In [13]:
pytorch_architecture = ArchitectureFromLibraryDescr(callable=Identifier("CPnetWrapper"), import_from="cpnet_wrapper")

### Create the model

In [14]:
from bioimageio.spec.model.v0_5 import LicenseId
from bioimageio.spec.common import HttpUrl

In [15]:
my_model_descr = ModelDescr(
    name="Cellpose Gold Nuclear 1135",
    description="An experimental Cellpose nuclear model fine-tuned on ovules 1136, 1137, 1139, 1170 and tested on ovules 1135. A model for BioImage.IO team to test and develop post-processing tools.",
    authors=[Author(name="Qin Yu", affiliation="EMBL", github_user="qin-yu")],
    cite=[CiteEntry(text="For more details see the manuscript", doi=Doi("10.1101/2024.02.19.580954"))],
    license=LicenseId("MIT"),
    documentation=HttpUrl("https://github.com/kreshuklab/go-nuclear/blob/main/README.md"),
    git_repo=HttpUrl("https://github.com/kreshuklab/go-nuclear"),
    inputs=[input_descr],
    outputs=[output_descr],
    weights=WeightsDescr(
        pytorch_state_dict=PytorchStateDictWeightsDescr(
            source=Path('/g/kreshuk/yu/temp/cp_wrapper_state_dict_1135_gold.pth'),
            architecture=pytorch_architecture,
            pytorch_version=pytorch_version
        ),
        torchscript=TorchscriptWeightsDescr(
            source=Path('/g/kreshuk/yu/temp/cp_wrapper_traced_1135_gold.pt'),
            pytorch_version=pytorch_version,
            parent="pytorch_state_dict", # these weights were converted from the pytorch_state_dict weights ones.
        ),
    ),
    )
print("created descr!")

2024-05-29 13:28:07.160 | Level 30 | bioimageio.spec._internal.field_warning:issue_warning:149 - documentation: No '# Validation' (sub)section found in https://github.com/kreshuklab/go-nuclear/blob/main/README.md.
2024-05-29 13:28:07.217 | Level 30 | bioimageio.spec._internal.field_warning:issue_warning:149 - covers: Failed to generate cover image(s): Failed to construct cover image from shape (32, 2, 224, 224)


created descr!


### Dynamic validation

In [16]:
from bioimageio.core import test_model
summary = test_model(my_model_descr)
summary.display()

2024-05-29 13:28:08.338 | INFO     | bioimageio.core._resource_tests:_test_model_inference:122 - starting 'Reproduce test outputs from test inputs'
2024-05-29 13:28:15.527 | INFO     | bioimageio.core._resource_tests:_test_model_inference_parametrized:192 - Testing inference with 4 different input tensor sizes



|        ✔️       | bioimageio validation passed |
|       ---       |             ---              |
| source          | in-memory                    |
| format version  | model 0.5.0                  |
| bioimageio.spec | 0.5.2post5                   |
| bioimageio.core | 0.6.5                        |

|  ❓  | location |                                detail                               |
| --- |   ---    |                                 ---                                 |
| ✔️  |          | initialized ModelDescr to describe model 0.5.0                      |
|     |          |                                                                     |
| ✔️  |          | Has expected resource type                                          |
|     |          |                                                                     |
| ✔️  |          | Reproduce test outputs from test inputs                             |
|     |          |                                                                     |
| ✔️  |          | Run inference for inputs with batch_size: 1 and size parameter n: 0 |
|     |          |                                                                     |


### Package the model

In [17]:
from bioimageio.spec import save_bioimageio_package

print("package path:", save_bioimageio_package(my_model_descr, output_path=Path('cellpose_gold_1135.zip')))

package path: cellpose_gold_1135.zip
